**Github API v4 preview using GraphQL**

Github's API is moving (slowly) over to GraphQL in place of their REST API. Given my prior exploratory analysis of Github's data [found here](https://github.com/EldritchJS/kono-notebook/blob/master/kono.ipynb) I set out to see what moving to the new API buys me, or loses me, as it were. Here's a few things that stand out:

1) Single Endpoint  

2) Query-driven syntax

The single endpoint trades looking up endpoint formats and return values for looking up query field and connector values. IMHO the latter is more advantageous, as endpoint syntax and formatting can get unruly in a hurry. Having a single endpoint lets data scientists focus on their queries, staying relegated to JSON. The queries follow (mostly) intuitive graph pattern wherein one traverses the docs/graphs until reaching the desired node(s).

    
In terms of connections, to get started, one need only generate a personal access token on Github's site. 

Programmitically, one needs the requests and json Python libraries imported, per the following. Additionally, the single endpoint is defined along with my Github authorization token (which is a whole bunch of Xs for what's in the repo, go get your own token please)

In [ ]:
import requests
import json

endpoint = 'https://api.github.com/graphql'
headers = {'Authorization': 'bearer PUTYOUROWNTOKENHERE'}


I'll start with a call to get the members of my team's Github organization. Note: there's a rate limiting factor of 100 for the number of members to returned. One can either get the first or last 100. Pagination is provided for larger groups. Also of interest: our mascot's account is spelled Radly, as opposed to Radley or Radlee.

In [117]:
query = ''' \
{ \
    organization(login: \"radanalyticsio\") { \
    members(first: 100) { \
      edges { \
        node { \
          name \
          avatarUrl \
        } \
      } \
    } \
  } \
} \
'''

In [ ]:
r = requests.post(endpoint, json.dumps({"query": query}), headers=headers)
print(json.dumps(r.json(), indent=4, sort_keys=True))

Next I can do a simple query of the first 100 repositories I own, listing the repo's name, disk usage, and number of forks.

In [ ]:
query=''' \
{ \
repositoryOwner(login : \"EldritchJS\") \
    {login repositories (first : 100) \
        {edges {node {name, diskUsage, forkCount }}} \
    } \
} \
'''

In [ ]:
r = requests.post(endpoint, json.dumps({"query": query}), headers=headers)
print(json.dumps(r.json(), indent=4, sort_keys=True))

Here's a call to get the members of our organization. Note: there's a rate limiting factor of 100 for the number of members to returned. One can either get the first or last 100. Pagination is provided for larger groups. Also of interest: our mascot's account is spelled Radly, as opposed to Radley or Radlee.

Once I have a repository I'm interested in, I can acquire data pertaining to said repository. In the next example I get a list of names of users who have starred the equoid-openshift repo, those who are watching the repo, and the number of issues. All fields of interest in my previous work on repository health metrics. 

In [120]:
query = ''' \
{ \
    repository(owner: \"EldritchJS\", name: \"equoid-openshift\") { \
    stargazers(first: 100) { \
        edges { \
            node { \
                name \
            } \
        } \
    } \
    watchers(first : 100) { \
        edges { \
            node { \
                name \
                } \
            } \
        } \
    issues( first: 100) { \
        edges { \
            node { \
                comments( first: 100) { \
                    edges { \
                        node { \
                        body \
                        } \
                    } \
                }\
                } \
            } \
        } \
    } \
} \
'''

In [121]:
r = requests.post(endpoint, json.dumps({"query": query}), headers=headers)
print(json.dumps(r.json(), indent=4, sort_keys=True))

{
    "data": {
        "repository": {
            "issues": {
                "edges": []
            }, 
            "stargazers": {
                "edges": [
                    {
                        "node": {
                            "name": "Jirka Kremser"
                        }
                    }
                ]
            }, 
            "watchers": {
                "edges": []
            }
        }
    }
}
